### Global Configurations

In [232]:
# Imports
import requests
import pandas as pd

# API variables
api_key = "RGAPI-cb7dc070-2672-45bc-ad67-fdfeaa63a8d4"
base_url = "https://americas.api.riotgames.com"

# Search parameters
gameName = "ARMAO"
tag_line = "NA1"

### Summoner Functions

In [279]:
# Function to get unique PUUID by summoner name
def get_puuid_by_name(gameName, tag_line):
    try:
        url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tag_line}"
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        if "puuid" not in data:
            raise KeyError("PUUID not found in response")
        return data["puuid"]
    except requests.exceptions.RequestException as e:
        print(f"Error making request for {gameName}#{tag_line}: {str(e)}")
        return None
    except (KeyError, ValueError) as e:
        print(f"Error parsing response for {gameName}#{tag_line}: {str(e)}")
        return None

def get_summoner_name_by_puuid(puuid):
    try:
        url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}"
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        response.raise_for_status()
        data = response.json()
        return data["gameName"]
    except requests.exceptions.RequestException as e:
        print(f"Error making request for PUUID {puuid}: {str(e)}")
        return None
    except ValueError as e:
        print(f"Error processing summoner data for PUUID {puuid}: {str(e)}")
        return None
        

# Function to get summoner data by PUUID
def get_summoner_by_puuid(puuid):
    try:
        if not puuid:
            raise ValueError("PUUID cannot be empty")
        url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        response.raise_for_status()
        data = response.json()
        if not data:
            raise ValueError("Empty response received")
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error making request for PUUID {puuid}: {str(e)}")
        return None
    except ValueError as e:
        print(f"Error processing summoner data for PUUID {puuid}: {str(e)}")
        return None

# Function to get rank data by PUUID
def get_rank_data_by_summoner_id(puuid):
    try:
        if not puuid:
            raise ValueError("PUUID cannot be empty")
        url = f"https://na1.api.riotgames.com/lol/league/v4/entries/by-puuid/{puuid}"
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        response.raise_for_status()
        data = response.json()
        if not isinstance(data, list):
            raise ValueError("Expected list response for rank data")
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error making request for PUUID {puuid}: {str(e)}")
        return None
    except ValueError as e:
        print(f"Error processing rank data for PUUID {puuid}: {str(e)}")
        return None

# Function to get champion mastery by PUUID and champion ID
def get_champion_mastery_by_puuid(puuid, champion_id):
    try:
        url = f"https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}/by-champion/{champion_id}"
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error making request for PUUID {puuid}: {str(e)}")
        return None
    except ValueError as e:
        print(f"Error processing champion mastery data for PUUID {puuid}: {str(e)}")
        return None


#### Summoner Functions Tests

In [281]:
# Test get_puuid_by_name function
def test_get_puuid_by_name():
    puuid = get_puuid_by_name(gameName, tag_line)
    
    # Check if we got a valid PUUID back
    if puuid and isinstance(puuid, str) and len(puuid) > 0:
        print(f"✅ Successfully retrieved PUUID for {gameName}#{tag_line}")
        print(f"PUUID: {puuid}")
        return True
    else:
        print(f"❌ Failed to retrieve valid PUUID for {gameName}#{tag_line}")
        return False
    
    
# Test get_summoner_name_by_puuid function
def test_get_summoner_name_by_puuid():
    puuid = get_puuid_by_name(gameName, tag_line)
    summoner_name = get_summoner_name_by_puuid(puuid)
    
    # Check if we got a valid summoner name back
    if summoner_name and isinstance(summoner_name, str) and len(summoner_name) > 0:
        print(f"✅ Successfully retrieved summoner name for PUUID: {puuid}")
        print(f"Summoner name: {summoner_name}")
        return True
    else:
        print(f"❌ Failed to retrieve valid summoner name for PUUID: {puuid}")
        return False



# Test get_summoner_by_puuid function
def test_get_summoner_by_puuid():
    puuid = get_puuid_by_name(gameName, tag_line)
    summoner_data = get_summoner_by_puuid(puuid)
    
    # Check if we got valid summoner data back
    if summoner_data and isinstance(summoner_data, dict) and 'id' in summoner_data:
        print(f"✅ Successfully retrieved summoner data for PUUID: {puuid}")
        print(f"Summoner data: {summoner_data}")
        return True
    else:
        print(f"❌ Failed to retrieve valid summoner data for PUUID: {puuid}")
        return False


# Test get_rank_by_summoner_id function
def test_get_rank_by_summoner_id():
    puuid = get_puuid_by_name(gameName, tag_line)
    rank_data = get_rank_data_by_summoner_id(puuid)
    
    # Check if we got valid rank data back
    if rank_data and isinstance(rank_data, list):
        print(f"✅ Successfully retrieved rank data for PUUID: {puuid}")
        print(f"Rank data: {rank_data}")
        return True
    else:
        print(f"❌ Failed to retrieve valid rank data for PUUID: {puuid}")
        return False

# Test get_champion_mastery_by_puuid function
def test_get_champion_mastery_by_puuid():
    puuid = get_puuid_by_name(gameName, tag_line)
    mastery_data = get_champion_mastery_by_puuid(puuid, 64)
    
    # Check if we got valid mastery data back
    if mastery_data and isinstance(mastery_data, dict) and len(mastery_data) > 0:
        print(f"✅ Successfully retrieved champion mastery data for PUUID: {puuid}")
        print(f"Mastery data: {mastery_data}")
        return True
    else:
        print(f"❌ Failed to retrieve valid champion mastery data for PUUID: {puuid}")
        return False

# Run the tests
print("Running get_puuid_by_name tests...")
test_get_puuid_by_name()
print("Running get_summoner_name_by_puuid tests...")
test_get_summoner_name_by_puuid()
print("\nRunning get_summoner_by_puuid tests...")
test_get_summoner_by_puuid()
print("\nRunning get_rank_by_summoner_id tests...")
test_get_rank_by_summoner_id()
print("\nRunning get_champion_mastery_by_puuid tests...")
test_get_champion_mastery_by_puuid()



Running get_puuid_by_name tests...
✅ Successfully retrieved PUUID for ARMAO#NA1
PUUID: -7A55h5Ap4Bw8XZCFryUhBjT4q8wiIcxR4U_bnCcErLc1Ry0zwHnlnajqNBH3nR-Wwvz5mHJKv9FRQ
Running get_summoner_name_by_puuid tests...
✅ Successfully retrieved summoner name for PUUID: -7A55h5Ap4Bw8XZCFryUhBjT4q8wiIcxR4U_bnCcErLc1Ry0zwHnlnajqNBH3nR-Wwvz5mHJKv9FRQ
Summoner name: ARMAO

Running get_summoner_by_puuid tests...
✅ Successfully retrieved summoner data for PUUID: -7A55h5Ap4Bw8XZCFryUhBjT4q8wiIcxR4U_bnCcErLc1Ry0zwHnlnajqNBH3nR-Wwvz5mHJKv9FRQ
Summoner data: {'id': 'SnTLNiQ-Kl1hkAZgS6Fn3ej6dRurY4JZKdHrHLaMJP6TBfM', 'accountId': 'ndxwVpNfC3DizeTq01amiKZ7bf1BKOR79GYVyTa2u1hAeSQ', 'puuid': '-7A55h5Ap4Bw8XZCFryUhBjT4q8wiIcxR4U_bnCcErLc1Ry0zwHnlnajqNBH3nR-Wwvz5mHJKv9FRQ', 'profileIconId': 6296, 'revisionDate': 1741556977617, 'summonerLevel': 433}

Running get_rank_by_summoner_id tests...
✅ Successfully retrieved rank data for PUUID: -7A55h5Ap4Bw8XZCFryUhBjT4q8wiIcxR4U_bnCcErLc1Ry0zwHnlnajqNBH3nR-Wwvz5mHJKv9FRQ


True

### Match Functions

In [239]:
# Function to get match history by PUUID
def get_matches_by_puuid(puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?count=50"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    matches = response.json()
    return matches

# Function to get match data by match ID
def get_match_data_by_match_id(match_id):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    match_data = response.json()
    return match_data

# Function to get the PUUIDs of the participants in a match
def get_match_puuids(match_id):
    try:
        match_data = get_match_data_by_match_id(match_id)
        if "metadata" not in match_data:
            raise KeyError("Match data does not contain metadata")
        if "participants" not in match_data["metadata"]:
            raise KeyError("Match metadata does not contain participants")
        return match_data["metadata"]["participants"]
    except Exception as e:
        print(f"Error getting match PUUIDs for match {match_id}: {str(e)}")
        return []

# Function to check if a match is a ranked match
def check_ranked(match_id):
    try:
        match_data = get_match_data_by_match_id(match_id)
        if "info" not in match_data:
            print(f"Match {match_id} data does not contain info")
            return False
        if "queueId" not in match_data["info"]:
            print(f"Match {match_id} info does not contain queueId")
            return False
        return match_data["info"]["queueId"] == 420
    except Exception as e:
        print(f"Error checking if match {match_id} is ranked: {str(e)}")
        return False

In [237]:
# Abstracted function to get ranked matches by summoner name
def get_ranked_matches_by_name(gameName, tag_line):
    puuid = get_puuid_by_name(gameName, tag_line)
    matches = get_matches_by_puuid(puuid)
    ranked_matches = []
    for match in matches:
        if check_ranked(match):
            ranked_matches.append(match)
    return ranked_matches

### Rest

In [285]:
# High-Level function to get the start data of a match
def get_match_start_data(match_id):
    players = []
    match_data = get_match_data_by_match_id(match_id)
    for participant in match_data["info"]["participants"]:
        participant_puid = participant["puuid"]
        rank = get_rank_data_by_summoner_id(participant_puid)[0]["tier"]
        champion = participant["championName"]
        champion_mastery = get_champion_mastery_by_puuid(participant_puid, participant["championId"])["championPoints"]
        role = participant["teamPosition"]
        players.append({
            "rank": rank,
            "champion": champion,
            "champion_mastery": champion_mastery,
            "role": role
        })
    return players
    

In [286]:
# Extracting a ranked match to run testing analysis on
ranked_matches = get_ranked_matches_by_name(gameName, tag_line)
test_match = ranked_matches[0]

In [287]:
print(get_match_start_data(test_match))

[{'summoner_name': '', 'rank': 'CHALLENGER', 'champion': 'Urgot', 'champion_mastery': 1526444, 'role': 'TOP'}, {'summoner_name': '', 'rank': 'DIAMOND', 'champion': 'LeeSin', 'champion_mastery': 785274, 'role': 'JUNGLE'}, {'summoner_name': '', 'rank': 'CHALLENGER', 'champion': 'Jayce', 'champion_mastery': 91729, 'role': 'MIDDLE'}, {'summoner_name': '', 'rank': 'CHALLENGER', 'champion': 'Varus', 'champion_mastery': 268467, 'role': 'BOTTOM'}, {'summoner_name': '', 'rank': 'GRANDMASTER', 'champion': 'Renata', 'champion_mastery': 21380, 'role': 'UTILITY'}, {'summoner_name': '', 'rank': 'GRANDMASTER', 'champion': 'Aatrox', 'champion_mastery': 267186, 'role': 'TOP'}, {'summoner_name': '', 'rank': 'CHALLENGER', 'champion': 'Vi', 'champion_mastery': 203139, 'role': 'JUNGLE'}, {'summoner_name': '', 'rank': 'CHALLENGER', 'champion': 'Chogath', 'champion_mastery': 18837, 'role': 'MIDDLE'}, {'summoner_name': '', 'rank': 'CHALLENGER', 'champion': 'Jinx', 'champion_mastery': 17038, 'role': 'BOTTOM'},

In [ ]:
# Getting the PUUIDs of the participants in a match
puids = get_match_puuids(test_match)

# Getting the rank of each player in a ranked match
for puid in puids:
    name = get_summoner_name_by_puuid(puid)
    rank = get_rank_data_by_summoner_id(puid)[0]["tier"]
    print(f"{name} is a {rank} player")

In [ ]:
test_match_data = get_match_data_by_match_id(test_match)

{'metadata': {'dataVersion': '2', 'matchId': 'NA1_5242178677', 'participants': ['qnizct8MH4s9QSjwzUtsShub66QAaVK5a6asnIXF-BBU_Bgo3agmv5olaufOENoswUJtepwz-XDEew', 'JDBdBiOylAk2VOrqNKrJY46VXlcIQqpNlQ5x7YIb3bFY6spThhQE7ypQ8BxbyD-s8FqV-kmXpSfsfQ', 'DUZ-zqC5sMi_4yjdU3xrGwkjd-DQn4sQf9Q6I7enm6HwABmCETNDR1-0GbVpKtPEmVrWpBHIxT5A3A', 'kHXMVkfR9xhCt6CDmisklYHV7yzjOwte9CEs-LZ6r1mbnoo5kEXwHy1ynuZRwPraQheWBMpsa4HNOg', 'qLeuqlce7pEHHjydbIcDWEVW9b-u56zninxl_Oz2t7fxQM1Y2W-o1dRJsdSPO3pvvvPkuecPX5Lf5w', 'UOwhXdY-QiWyuXBWrTfWrDyykzcZjyvDlHOK2GateQ-5kP8MkNaQ_DzaRGVPVcP3HuDtQRXCHs-EQA', '-7A55h5Ap4Bw8XZCFryUhBjT4q8wiIcxR4U_bnCcErLc1Ry0zwHnlnajqNBH3nR-Wwvz5mHJKv9FRQ', 'KdsueFZIQY5bAdmPSs08nIFpIQQKvkaLRCKgov4zggAAsdIT4iGQppiGyJ3I5XngU8YeAyDqs_7gZA', 'y6BOBR8PBy3pYzArIMOM9Hs0D-kJ-UcxN5EAYyTHLmF2jizoHwIK-yxcJlVN9vEYw1HxRit41326Sg', 'EkzCEly_c8NdUpD8ErFN3Y02CnSuo8EFshNNBOePfzPmf4LmeFcxbvmILw0X32jRNhys6aum1odLBw']}, 'info': {'endOfGameResult': 'GameComplete', 'gameCreation': 1741223410375, 'gameDuration': 1437, '

In [282]:
# Extracting the participants roles and champions
test_match_info = test_match_data["info"]
test_match_info_participants = test_match_info["participants"]

for participant in test_match_info_participants:
    summoner_name = get_summoner_name_by_puuid(participant["puuid"])
    position = participant["teamPosition"]
    champion = participant["championName"]
    champion_mastery = get_champion_mastery_by_puuid(participant["puuid"], participant["championId"])["championPoints"]
    print(f"{summoner_name} played {champion} in {position} with {champion_mastery} champion points")


quante played Urgot in TOP with 1526444 champion points
An Nuo l played LeeSin in JUNGLE with 785274 champion points
Reppy played Jayce in MIDDLE with 91729 champion points
SnowyStepZ played Varus in BOTTOM with 268467 champion points
ttv Skyward played Renata in UTILITY with 21380 champion points
Maguyusi played Aatrox in TOP with 267186 champion points
ARMAO played Vi in JUNGLE with 203139 champion points
搞事就报警把你妈妈捉走 played Chogath in MIDDLE with 18837 champion points
Alpha Smoker played Jinx in BOTTOM with 17038 champion points
JustAChillZilean played Braum in UTILITY with 34761 champion points
